## Exercise (Intro to Machine Learning: 2 of 7)
Basic Data Exploration

### Think About Your Data
The newest house in your data isn't that new. A few potential explanations for this:

1. They haven't built new houses where this data was collected.
2. The data was collected a long time ago. Houses built after the data publication wouldn't show up.

If the reason is explanation #1 above, does that affect your trust in the model you build with this data? What about if it is reason #2?

How could you dig into the data to see which explanation is more plausible?

Check out this [discussion](https://www.kaggle.com/learn-forum/60581) thread to see what others think or to add your ideas.

> → EDA라는 단계에서 뭘해야 되는 지 알려주는 부분. 말그대로 train/test 데이터에 대해 탐색적으로 분석하는 단계라고 보면 되는 것 같다.

>데이터 자체로는 많은 것들을 알려주는 거라고 착각할 수 있지만 자세히 보면, 가장 최근에 지어진 건물 년도 수가 2010년 인것을 보니 최근(2021년) 데이터도 아닐 수도 있다.
>> **Mohammad Shariatmadari**의 comment에 따르면: 
1. 가장 활발하게 거래가 이루어지는 특정 month가 1년마다 같은 (패턴으로) 반복되는 것을 확인할 수 있었으며
2. 연도별 거래 건수를 봤을 때는 가장 마지막 년도인 2010년의 거래 데이터 조차 다른 년도에 비해 부족한 것을 보니 특정 시점 부터의 데이터가 없다고 보면 될 것 같다.



## Exercise (Intro to Machine Learning: 3 of 7)
Your First Machine Learning Model

```python
ex_data = pd.read_csv('...')

# Prediction Target (y)
y = ex_data.target_col # dot notation (when you call 1 column)

# Features (X)
feature_names = ['a_col', 'b_col', ... , 'z_col']
X = ex_data[feature_names]

# specify Model
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=1)

# Fit the Model
model.fit(X,y)

# make Prediction
print(model.predict(X))
```

## Exercise (Intro to Machine Learning: 4 of 7)
Model Validation
- to measure the quality of model
- to iteratively improve models

(Test accuracy 예측, Overfitting 방지)

### What is Model Validation
사람들이 가장 많이 하는 실수 → 예측 정확도를 평가할 때

> ∴ train data로 예측하고 train data의 target과 비교하기 때문

모델의 성능 평가지표(metrics) 중 MAE(Mean Absolute Error)가 있다.
```
error = actual - predicted
```
즉, MAE 측정기준은 각 오차의 절대값들의 평균이다. (낮을 수록 좋은 모델)
```python
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)
```

### The Problem with "In-Sample" Scores
train data로 예측한 값을 train data의 target과 비교하는 것은 아주 위험한 실수.

왜냐하면 train data가 특정 변수에 대해 편향적인 값을 가지고 있는 데 new data의 해당 변수는 전혀 다른 경향을 가지고 있을 때 정확한 결과를 예측할 수 없기 때문.

이것을 방지하기 위해서, model설계 과정에서 특정 data를 배제시키고, excluded data는 새로 맞닥트릴 unseen data에 대한 정확도를 평가하기 위해 쓰인다. 이것을 **validation data**라고 한다.

> **validation data**: 모델의 성능을 평가하기 위한 data set 
>> 결국에는 overfitting을 방지 하기 위함

> - Test data: 모델의 '최종 성능'을 평가하기 위해 쓰임  
- Validation data: 여러 모델 중에 최종 모델을 선정하기 위한 성능 평가에 관여됨.

※ 만약 improve model 하고 싶다면:  
- features를 바꾸거나
- model을 바꾸면 된다.

``` python
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0) # train, val 분리를 항상 같게 하기 위한 random_state

# Define model
melbourne_model = DecisionTreeRegressor(random_state=0) # model의 결과를 항상 같게 하기 위한 random_state
# Fit model (모델 학습은 train data로)
melbourne_model.fit(train_X, train_y)

# get predicted prices on validation data (모델 평가는 val data로)
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))
```

## Exercise (Intro to Machine Learning: 5 of 7)
### Underfitting and Overfitting

`Scikit-learn module` 에서 제공하는 `DecisionTree Model`을 사용하게 되면 split[분기점]이 늘어나면 늘어날수록 수많은 그룹들이 생겨난다. 그룹이 많아진다는 얘기는 그 그룹에 속하는 데이터 수는 점점 줄어든다는 얘기다. 이렇게 적어진 데이터로 target value를 예측하게 되면 학습 데이터 양이 줄어들기 때문에 새로운 data에 대해서 잘못 예측할 수 있는 위험이 있다. 이러한 현상을 **Overfitting**이라고 한다.
> Overfitting은 train data에 대해선 거의 완벽히 예측 가능하지만 validation data나 new data에 대해서는 아주 하찮은 성과를 보여줄 수도 있다.

또한 만약 Decision Tree의 분기점을 아주 적게 설정한다면, 그룹에 속하는 데이터 양은 많아질 지 몰라도 분류의 기준이 모호해서 train data조차 제대로 분류하지 못할 수 있다. 이러한 현상을 **Underfitting**이라고 한다. 이것은 model이 중요한 구분점이나 패턴을 파악하지 못했을 때 일어나게 된다.

### 그렇다면 최적의 모델을 찾는 방법은?
우리가 원하는 모델은 accuracy가 Overfitting과 Underfitting 사이에 있는 지점의 모델이다. 예를 들어 DecisionTree의 옵션 중에는 max_leaf_nodes라는 argument가 있다. 이 옵션은 모형의 잎(즉, 최종 그룹) 수를 조절하여 Overfit과 Underfit을 조절하는데, 잎의 수가 많을 수록 Underfit → Overfit으로 이동한다고 보면 된다.
![image](http://i.imgur.com/2q85n9s.png)
그림에서는 MAE를 모델 성능평가지표로 쓰였는데 낮을 수록 정확도가 높다고 판단되므로 느낌표 위치, 즉 Validation이 가장 낮은 위치에 있을 때가 가장 정확도가 높게 되는 최적의 모델이라고 볼 수 있다.

이를 위해서
```python
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
```
이렇게 get_mae함수를 만들게 되면
```python
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))
```
이런식으로 잎의 수를 몇개로 정해야 최적의 모델을 가질 수 있는 지 확인해볼 수 있다.

### Conclusion
> - **Overfitting**: capturing spurious patterns that won't recur in the future, leading to less accurate predictions, or
- **Underfitting**: failing to capture relevant patterns, again leading to less accurate predictions.

In [ ]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

# 내가 푼 방법
optimal_max_leaf_nodes = 0
for max_leaf_nodes in candidate_max_leaf_nodes:
    if optimal_max_leaf_nodes == 0:
        optimal_max_leaf_nodes = max_leaf_nodes
    if min(get_mae(optimal_max_leaf_nodes, train_X, val_X, train_y, val_y),
           get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)) == get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
        optimal_max_leaf_nodes = max_leaf_nodes

best_tree_size = optimal_max_leaf_nodes

# 더 깔끔한 방법
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
# 한줄 for문 {입력될 형식 for i in iterable}
best_tree_size = min(scores, key=scores.get)
# min, max는 기본적으로 dict 변수에 대해 key값을 비교 후 key값을 return
# 하지만 key=dict.get옵션을 추가하면 value값을 비교 후 key값을 return

'''--------------------------------------------------------------'''

# Fill in argument to make optimal size and uncomment
final_model = DecisionTreeRegressor(max_leaf_nodes = best_tree_size)

# fit the final model and uncomment the next two lines
final_model.fit(X, y) # final 모델에 대해서는 split data가 아닌 전체 data로 fit해야 됨

## Exercise (Intro to Machine Learning: 6 of 7)
### Random Forest
**Random Forest**는 Decision Tree의 Overfit, Underfit 방지를 위해 더 나은 성능을 낼 수 있는 모델 중 하나이다.
```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))
```
기계 학습에서의 Random forest[링크 텍스트](https://)는 분류, 회귀 분석 등에 사용되는 앙상블 학습 방법의 일종으로, 훈련 과정에서 구성한 다수의 결정 트리로부터 부류(분류) 또는 평균 예측치(회귀 분석)를 출력함으로써 동작한다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state = 1)

# fit your model
rf_model.fit(train_X,train_y)

# Calculate the mean absolute error of your Random Forest model on the validation data
val_pred = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(val_pred, val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

[제출 완료!](https://www.kaggle.com/c/home-data-for-ml-course/submissions)